In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle 
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from joblib import dump
from joblib import load
from flask import Flask
import numpy as np
import pandas as pd


In [2]:
app = FastAPI()

In [3]:
cat_model = load("Cat_model.joblib","rb")

In [4]:
print("CatBoost Prediction:", cat_model.predict(['0', '2', '1', '0', '4', '0', '1']))

CatBoost Prediction: 0


In [5]:
print("CatBoost Prediction:", cat_model.predict([1, 0, 4, 0, 0, 3,3]))

CatBoost Prediction: 1


In [6]:
print("CatBoost Prediction:", cat_model.predict([1, 0, 3, 4, 0, 3,3]))

CatBoost Prediction: 0


In [7]:
origins = ["*"]

app.add_middleware(
CORSMiddleware,
allow_origins=["*"], # Allows all origins
allow_credentials=True,
allow_methods=["*"], # Allows all methods
allow_headers=["*"], # Allows all headers
)

In [5]:
class model_input(BaseModel):
    age_group: str  
    income_group: str
    loan_amount_group:str  
    loan_percent_income_group:str
    loan_intent:str   
    person_home_ownership:str
    person_emp_length:str

In [8]:
cat_model = load('CatBoost_model.joblib',"rb")

In [6]:
@app.post('/loan_approval_prediction')
def loan_pred(input_parameters: model_input): 
    
    input_data = input_parameters.json()

    #userdata = input_data.to_dict()
    # will be a list: [{"Firstname":"Joaquin", "Surname": "Phoenix","Age": 49}]
    #userdata = userdata[0] # take the first item from the list
    # will be a dictionary: {"Firstname":"Joaquin", "Surname": "Phoenix","Age": 49}

    input_dictionary = json.loads(input_data)# input_data
    
    
    age = input_dictionary['age_group']
    income = input_dictionary['income_group']
    loan = input_dictionary['loan_amount_group']
    loan_percent = input_dictionary['loan_percent_income_group']
    intent = input_dictionary['loan_intent']
    person_home_ownership = input_dictionary['person_home_ownership']
    emp_length = input_dictionary['person_emp_length']
    
    input_list = [age , income, loan, loan_percent , intent ,person_home_ownership , emp_length]
    
    prediction = cat_model.predict([input_list])
    
    if (prediction[0] == 0):
        return 'Congratulations, you are eligible to get a loan'
    else:
        return 'Sorry, You are Not eligible to get a loan.'
    
     


In [ ]:
ngrok_tunnel = ngrok.connect(80)
print("Public URL:", ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

Public URL: https://2663-2a02-ce0-1000-2848-3086-a83f-9e42-4e7b.ngrok-free.app


INFO:     Started server process [368]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)


INFO:     2a02:ce0:1000:2848:3086:a83f:9e42:4e7b:0 - "POST /loan_approval_prediction HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:ce0:1000:2848:3086:a83f:9e42:4e7b:0 - "POST /loan_approval_prediction HTTP/1.1" 200 OK
INFO:     2a02:4780:11:1503:0:622:9632:3:0 - "GET /display_message HTTP/1.1" 200 OK
INFO:     2a02:ce0:1000:2848:3086:a83f:9

In [8]:
class ModelInput(BaseModel):
    age_group: str  
    income_group: str
    loan_amount_group: str  
    loan_percent_income_group: str
    loan_intent: str   
    person_home_ownership: str
    person_emp_length: str

cat_model = load('CatBoost_model.joblib')

@app.post('/loan_approval_prediction')
def loan_pred(input_parameters: ModelInput): 
    age = input_parameters.age_group
    income = input_parameters.income_group
    loan = input_parameters.loan_amount_group
    loan_percent = input_parameters.loan_percent_income_group
    intent = input_parameters.loan_intent
    person_home_ownership = input_parameters.person_home_ownership
    emp_length = input_parameters.person_emp_length
    
    input_list = [age, income, loan, loan_percent, intent, person_home_ownership, emp_length]
    
    prediction = cat_model.predict([input_list])[0]
    
    if prediction[0] == 0:
        return 'Congratulations, you are eligible to get a loan'
    else:
        return 'Sorry, you are not eligible to get a loan.'
    
    
    response_data = {
        'prediction': prediction[0],
        'message': 'Congratulations, you are eligible to get a loan' if prediction[0] == 0 else 'Sorry, you are not eligible to get a loan.'
    }

    # Save the response in a file
    with open('loan_response.json', 'w') as file:
        json.dump(response_data, file)

    return 'Response saved successfully.'



In [9]:
@app.get('/display_message')
def display_message():
    try:
        # Load the saved response from the file
        with open('loan_response.json', 'r') as file:
            response_data = json.load(file)

        # Get the message from the response data
        message = response_data['message']

        return message
    except FileNotFoundError:
        return 'Response not found.'

In [11]:
class ModelInput(BaseModel):
    age_group: str  
    income_group: str
    loan_amount_group: str  
    loan_percent_income_group: str
    loan_intent: str   
    person_home_ownership: str
    person_emp_length: str

cat_model = load('CatBoost_model.joblib')

app = FastAPI()

@app.post('/loan_approval_prediction')
def loan_pred(input_parameters: ModelInput): 
    age = input_parameters.age_group
    income = input_parameters.income_group
    loan = input_parameters.loan_amount_group
    loan_percent = input_parameters.loan_percent_income_group
    intent = input_parameters.loan_intent
    person_home_ownership = input_parameters.person_home_ownership
    emp_length = input_parameters.person_emp_length
    
    input_list = [age, income, loan, loan_percent, intent, person_home_ownership, emp_length]
    
    prediction = cat_model.predict([input_list])[0]
    
    response_data = {
        'prediction': prediction.item(),  # Convert int64 to int
        'message': 'Congratulations, you are eligible to get a loan' if prediction == 0 else 'Sorry, you are not eligible to get a loan.'
    }

    # Save the response in a file
    with open('loan_response.json', 'w') as file:
        json.dump(response_data, file)

    return response_data

@app.get('/display_message')
def display_message():
    try:
        # Load the saved response from the file
        with open('loan_response.json', 'r') as file:
            response_data = json.load(file)

        # Get the message from the response data
        message = response_data['message']

        return response_data
    except FileNotFoundError:
        return 'Response not found.'

In [ ]:
display_message()